## Introduction

* #TODO: INSERT LINK TO WANDB VIEW

## Setup

#### Install dependencies

* **torch**: PyTorch framework for the creation of neural networks
* **lightning**: Lightning wrapper for pytorch for simple network training
* **huggingface_hub**: HuggingFace hub for downloading word vectors
* **datasets**: HuggingFace datasets to download and load the data set
* **wandb**: Weights & Biases for experiment tracking
* **fasttext**: Word embedding library
* **nltk**: Natural Language Toolkit used for word tokenization
* **torchmetrics**: Extension to lightning to compute model metrics

In [1]:
import sys

import numpy as np
from unicodedata import bidirectional

%pip install -q torch lightning huggingface_hub datasets wandb nltk torchmetrics

if sys.platform == 'win32': # Windows requires different fasttext implementation
    %pip install -q fasttext-wheel
else: 
    %pip install -q fasttext

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Do all the imports needed in this project, but keep them separate depending on the part of the notebook that uses them. Reason: Makes reading easier in later sections if done subsequently, but don't users to import everything if they just want to run some specific cells

##### pytorch & lightning

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import lightning as L
from lightning.pytorch.loggers import WandbLogger
import torchmetrics

##### Huggingface

In [3]:
from huggingface_hub import hf_hub_download
from datasets import Dataset as HFDataset
from datasets import load_dataset

##### Tokenization & Embeddings

In [4]:
import nltk
import fasttext

##### Python Builtins

In [5]:
from typing import Callable
import time

##### Experiment Tracking

In [6]:
import wandb

#### Load dataset

Use the pre-defined method to load the dataset and do the train and validation split

In [7]:
train: HFDataset = load_dataset("tau/commonsense_qa", split="train[:-1000]")
valid: HFDataset = load_dataset("tau/commonsense_qa", split="train[-1000:]")
test: HFDataset = load_dataset("tau/commonsense_qa", split="validation")

print(len(train), len(valid), len(test))

8741 1000 1221


#### Setup Weights & Biases

Login to weights and biases to enable experiment tracking

In [8]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: schurtenberger-david (david-schurtenberger) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Preprocessing

#### Vocabulary/Embedding

* I decided to use the **FastText** library for this project, since in class it was said that FastText is superior to the other embedding models and there is no problem with embedding unknown words, because it can create word vectors from their subwords. Furthermore, I will be working with the **facebook/fasttext-en-vectors** word vectors from the HuggingFace hub. They embed words from the English language, which is the only relevant language.

* This choice influences decisions in the following pre-processing steps.

#### Format cleaning (e.g. html-extracted text)

* No format cleaning is performed, because we work with a carefully assembled and standardized dataset used in model benchmarking.

#### Tokenization

* *word_tokenizer* from the **nltk** library will be used. This tokenizer works well for the English language. It also splits punctuation from text, which matches the tokens the fasttext word vectors were trained on.

#### Lowercasing, stemming, lemmatizing, stopword/punctuation removal

* **Lowercasing**: Although the word vectors in use were trained on case-sensitive data, the tokenized words will be lowercased to reach a smaller vocabulary and minimize out-of-vocabulary words.
* **Stemming**: The word embedding model was not trained on word stems and therefore no stemming is carried out.
* **Lemmatizing**: The word tokens to be embedded will not be lemmatized, because the fasttext model was trained on un-lemmatized words and the n-gram encoding of the words used in fasttext preserves sub-word information.
* **Stopword/Punctudation removal**: Since the task is to answer common-sense questions, stopwords and punctuation will not be removed. Most of the questions are quite short and the loss of information if either a critical stopword in the question or punctuation that changes the meaning of the question is removed could be significant.

#### Removal of unknown/other words

* Since I am working with a fasttext model, the removal of unknown words is not necessary, because vectors for them can implicitly be built from their n-gram vectors. Also, the encounter of unknown words is not expected.

#### Truncation

* Input will not be truncated. After some data review, the question yielding the most embedded word vectors yields a tensor of shape **300x67**. Depending on the input format, the RNN model will have to perform significantly less than 100 time steps for the longest input, which is deemed to be feasible. Also, if padding is implemented correctly, for every timestep only the necessary amount of time steps will be executed.

#### Feature selection

* Of the available features, the **question**, the **choices** and the **answerKey** were chosen. While the *questionConcept* seemed like an interesting feature at first, after some data review it was determined, that this feature often simply contains a word from the question. In the end this feature was left out in order not to give too much emphasis to a single word that does likely not help answering the question at all.

#### Input format: how is data passed to the model?

###### Classifier

   * I chose the input for the *Classifier Model* to be a tensor of size **1800**. The first 300 elements are the averages of the embedded question tokens, next are 300 elements for every embedded and averaged answer vector from answer option 'A' to 'E'.
      * The average of the question vectors was chosen, because it is a good tradeoff between information retention and input  dimension for the classifier.
      * The question vector is before the answer vectors, because "Q&A" also has question first, then answers.
      * The answers are arranged from 'A' to 'E' because of alphabetical order.
      * The average of the answer embeddings has been chosen, since answers can consist of multiple words and therefore may yield multiple embedding vectors.

###### RNN + Classifier

   * I chose the input for the *RNN + Classifier Model* to be tensor of size **300 x (N + 10)**. The first *N* columns of the tensor are the word-embeddings of the question. The last *10* columns are the averages of the word embeddings for each answer choice, separated by a *SEP* token.
      * As separation token the character **¦** was chosen, because it is known to the embeddings model, but does not appear in the data.
      * The separation token was introduced to signal to the model, that after the input after this reserved token is an answer choice.
      * All answer embeddings are concatenated to the question embeddings to only need one full pass through the RNN model to get a prediction.
      * The answer embeddings are after the question embeddings, because "Q&A" also has question first, then answer.

#### Label format: what should the model predict?

* Both model architectures will predict a vector of length 5. Every answer choice ('A' through 'E') is encoded on an index in the vector (0 through 4). Since the classifier at the last stage of the model predicts the likelihood of each output, this output format seems the most reasonable. Also, with this kind of output, the model only needs to run once for every classification, which should increase compute performance.

#### Train/valid/test splits

* As seen in the *Introduction* section, the train/validation/test splits are performed as defined in the course

#### Batching, padding

* For each model, before training the batch size is determined at which 80% of GPU memory is being used. This threshold was chosen, because the GPU is well utilized, but there is still room for unforseen memory usage.
    * Note: For the classifier model, the entire training data sit would fit on the GPU memory at once. The batch size for that model will be statically set at **512**. Reason: Still use stochastic gradient descent with around 16 batches in total.
* Only the input of the **RNN + Classifier** model is of variable size, therefore padding is only necessary for that model. #TODO: specify padding for RNN model and check entire documentation of preprocessing, add documentation on batch size experiment

### Tokenize

Create method to tokenize and lowercase a given text

In [9]:
nltk.download("punkt_tab")

def tokenize(text: str) -> list[str]:
    return [w.lower() for w in nltk.word_tokenize(text, language="english")]

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


### Word embeddings

Download the english fasttext word vectors and load their model into the variable *wv_model*

Create a function to embed a list of tokenized words and return them as a list of pytorch tensors

In [10]:
model_path = hf_hub_download("facebook/fasttext-en-vectors", "model.bin")
wv_model = fasttext.load_model(model_path)

def get_embeddings_for_tokens(tokens: list[str]):
    return torch.stack([torch.tensor(wv_model[t]) for t in tokens])
    

### Data Loading and Formatting

Create a **pytorch** *Dataset* class in which the HuggingFace dataset is loaded and preprocessed. This allows for an easy integration with a *DataLoader* afterward.

In [11]:
TransformMethod = Callable[[torch.Tensor, list[torch.Tensor]], torch.Tensor | nn.utils.rnn.PackedSequence]

Create a separator token from a character that is known to the word vector model, but unused in the train, valid and test datasets 

In [12]:
def char_not_in_huggingface_dataset(char: str, dataset: HFDataset) -> bool:
    for datapoint in dataset:
        if char in datapoint["question"] or any(char in c for c in datapoint["choices"]["text"]):
            return False
    return True
    

separator = "¦"
assert separator in wv_model # Check if placeholder is a known token in the model
assert char_not_in_huggingface_dataset(separator, train)
assert char_not_in_huggingface_dataset(separator, valid)
assert char_not_in_huggingface_dataset(separator, test)

SEP_TOKEN = torch.tensor(wv_model[separator])
SEP_TOKEN.shape

torch.Size([300])

In [13]:
KEY_INDEX_MAPPING = {
    "A": 0,
    "B": 1,
    "C": 2,
    "D": 3,
    "E": 4,
}

class CommonsenseQADataset(Dataset):    
    def __init__(self, dataset: HFDataset, transform: TransformMethod):
        self.dataset: list[dict[str, torch.tensor | list[torch.tensor]]] = []
        self._target_transform = transform
        self._transform_hugging_face_dataset(dataset)
    
    def _transform_hugging_face_dataset(self, dataset: HFDataset):
        self.dataset.extend([{
            "feature": self._target_transform(
                get_embeddings_for_tokens(tokenize(entry["question"])),
                [get_embeddings_for_tokens(tokenize(choice)) for choice in entry["choices"]["text"]],
            ),
            "label": torch.eye(5)[KEY_INDEX_MAPPING[entry["answerKey"]]],
        } for entry in dataset])
        if len(self.dataset) != len(dataset):
            raise RuntimeError("Converted dataset is not full reflection of source data")
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        data_point = self.dataset[idx]
        return data_point["feature"], data_point["label"]

Transform function for **Classifier** network

* Function input: Question-tensor (300 x N) and answer-tensor (300 x 5)
* Function output: Classifier vector (1800)
    * Average of question vectors yields vector of size 300
    * Question vector and answer vectors are concatenated (5 * 300 = 1800)

In [14]:
def classifier_target_transform(question: torch.Tensor, answers: list[torch.Tensor]) -> torch.Tensor:
    return torch.cat((question.mean(dim=0), *(a.mean(dim=0) for a in answers)))

Transform function for **RNN + Classifier** network #TODO: update doc

* Function input: Question-tensor (300 x N) and answer-tensor (300 x 5)
* Function output: RNN-tensor (300 x (N + 10)) with N question vectors, 5 answer vectors and 5 SEP_TOKEN vectors separating the question from the answer and the answers from each other.

In [15]:
def rnn_target_transform(question: torch.Tensor, answers: list[torch.Tensor]) -> torch.Tensor:
    concatenated = [torch.cat((question, SEP_TOKEN.unsqueeze(0), answer.mean(dim=0).unsqueeze(0))) for answer in answers]
    return torch.stack(concatenated, dim=0)

Define collate function that implements the padding for the RNN+Classifier network

In [16]:
def rnn_collate_fn(batch: list[tuple[torch.Tensor, torch.Tensor]]):
    questions, labels = zip(*batch)  # Separate questions and labels
    
    # Find max sequence length in batch
    max_len = max(q.shape[1] for q in questions)

    # Pad all sequences to max_len
    padded_questions = []
    lengths = []
    
    for q in questions:
        seq_len = q.shape[1]  # Original sequence length
        lengths.append(seq_len)
        
        # Pad sequence to (5, max_len, 300)
        pad_size = (0, 0, 0, max_len - seq_len)  # (left_pad, right_pad) per dimension
        padded_q = torch.nn.functional.pad(q, pad_size)  # Pad along sequence_length
        padded_questions.append(padded_q)
    
    # Convert to tensor (batch_size, 5, max_len, 300)
    padded_questions = torch.stack(padded_questions)
    
    # Sort by sequence length (required for pack_padded_sequence)
    lengths, perm_idx = torch.tensor(lengths).sort(descending=True)
    padded_questions = padded_questions[perm_idx]
    labels = torch.stack(labels)[perm_idx]
    
    return padded_questions, lengths, labels

Transform HuggingFace datasets to pytorch Datasets for both Classifier and RNN+Classifier models

In [17]:
train_classifier = CommonsenseQADataset(train, classifier_target_transform)
valid_classifier = CommonsenseQADataset(valid, classifier_target_transform)
test_classifier = CommonsenseQADataset(test, classifier_target_transform)


train_rnn = CommonsenseQADataset(train, rnn_target_transform)
valid_rnn = CommonsenseQADataset(valid, rnn_target_transform)
test_rnn = CommonsenseQADataset(test, rnn_target_transform)

The following routine is used to determine the best batch size. The model is loaded

In [18]:
def batch_size_finder(data: Dataset, model: nn.Module, collate_fn = None, max_memory_usage=0.8) -> tuple[int, float]:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if not device == "cuda":
        raise RuntimeError("Can only be run on gpu")
    batch_size, memory_usage = 16, 0
    model.to(device)
    
    while True:
        loader = torch.utils.data.DataLoader(data, batch_size=batch_size, collate_fn=collate_fn, num_workers=4)
        inputs = nn.utils.rnn.PackedSequence(torch.tensor(()), torch.tensor(())) if collate_fn else torch.tensor([])
        labels = torch.tensor([])
        for i, l in loader:
            if isinstance(i, nn.utils.rnn.PackedSequence):
                if i.batch_sizes.shape > inputs.batch_sizes.shape:
                    inputs, labels = i, l                  
            elif i.shape > inputs.shape:
                inputs, labels = i, l
        try:
            inputs, labels = inputs.to(device), labels.to(device)
            _ = model(inputs)
            
            # Check memory usage
            mem_allocated = torch.cuda.memory_allocated(device)
            mem_reserved = torch.cuda.memory_reserved(device)
            mem_usage = mem_allocated / mem_reserved

            print(f"Batch Size: {batch_size}, Memory Usage: {mem_usage:%}")

            if mem_usage <= memory_usage:
                return batch_size // 2, memory_usage
            
            if mem_usage >= max_memory_usage:
                return batch_size, mem_usage
           
            batch_size *= 2
            memory_usage = mem_usage
        
        except RuntimeError as e:
            if 'out of memory' in str(e):
                return batch_size // 2, memory_usage
            raise e
    

Determine the optimal batch size for both Classifier and RNN+Classifier models. Optimal batch size was chosen to be when 80% of memory is used.

In [19]:
# HIDDEN_DIM_CLASSIFIER = 4096
# BATCH_SIZE_CLASSIFIER, memory_usage = batch_size_finder(train_classifier, Classifier(hidden_dim=HIDDEN_DIM_CLASSIFIER))
# print(f"Batch Size: {BATCH_SIZE_CLASSIFIER} @ {memory_usage:%} memory usage")
BATCH_SIZE_CLASSIFIER = 512

In [20]:
#HIDDEN_DIM_RNN_LINEAR, HIDDEN_DIM_RNN_GRU, BIDIRECTIONAL = 2048, 256, True
# BATCH_SIZE_RNN, memory_usage = batch_size_finder(train_rnn, RNNClassifier(hidden_dim_linear=HIDDEN_DIM_RNN_LINEAR, hidden_dim_gru=HIDDEN_DIM_RNN_GRU, bidirectional=BIDIRECTIONAL), rnn_collate_fn)
# print(f"Batch Size: {BATCH_SIZE_RNN} @ {memory_usage:%} memory usage")
BATCH_SIZE_RNN = 512

Instantiate data loaders for both model types

In [21]:
train_loader_classifier = torch.utils.data.DataLoader(train_classifier, batch_size=BATCH_SIZE_CLASSIFIER, shuffle=True, num_workers=4)
valid_loader_classifier = torch.utils.data.DataLoader(valid_classifier, batch_size=BATCH_SIZE_CLASSIFIER, num_workers=4)
test_loader_classifier = torch.utils.data.DataLoader(test_classifier, batch_size=BATCH_SIZE_CLASSIFIER, num_workers=4)

train_loader_rnn = torch.utils.data.DataLoader(train_rnn, batch_size=BATCH_SIZE_RNN, collate_fn=rnn_collate_fn, shuffle=True, num_workers=4)
valid_loader_rnn = torch.utils.data.DataLoader(valid_rnn, batch_size=BATCH_SIZE_RNN, collate_fn=rnn_collate_fn, num_workers=4)
test_loader_rnn = torch.utils.data.DataLoader(train_rnn, batch_size=BATCH_SIZE_RNN, collate_fn=rnn_collate_fn, num_workers=4)

# 1. Architecture: WordEmbeddings &rarr; Classifier

## Model

I chose to use **lightning** to create a streamlined model training process. The *LightningModule* subclass was created with the help of the [API doc](https://lightning.ai/docs/pytorch/LTS/common/lightning_module.html#lightningmodule-api) and the "experiment_tracking" notebook that we looked at in the Project Discussion lecture

The model architecture complies with the required architecture in the project description.
* Between input and hidden layer there is RELU-non-linearity as activation function. Reason: required
* The output of the second layer is activated using SoftMax. Reason: meaningful output activation for multiclass classification
* The metrics "val_loss", "val_acc", "train_loss", "train_acc" are logged after every epoch. Reason: Meaningful metrics, not overwhelming experiment tracking view #TODO: lightning part has moved

In [22]:
class Classifier(nn.Module):
    def __init__(
            self, 
            input_dim: int = 1800, 
            hidden_dim: int = 2048, 
            output_dim: int = 5, 
            dropout: float = 0.0
    ):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x: torch.Tensor, _=None) -> torch.Tensor:
        out = F.relu(self.fc1(x))
        out = self.dropout(out)
        return self.fc2(out)
    
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()

## Training

#TODO: training documentation head

Crete a LightningModule subclass that takes the model and all hyperparameters as arguments. With this class training can be simplified using the pytorch lightning framework. This class will also be reused for the RNN+Classifier model.


In [23]:
class CqaModule(L.LightningModule):
    def __init__(
            self, 
            model: Classifier,
            loss_function,
            output_dim=5,
            **kwargs        
    ):
        super().__init__()
        self.save_hyperparameters(kwargs)
        self.model = model
        self.model.reset_parameters()
        
        self.loss_fn = loss_function()
        
        self._train_acc = torchmetrics.Accuracy("multiclass", num_classes=output_dim)
        self._train_loss = []
        self._valid_acc = torchmetrics.Accuracy("multiclass", num_classes=output_dim)
        self._valid_loss = []
        self._max_valid_acc = 0
        
    def forward(self, x: torch.Tensor, lengths: torch.Tensor = None) -> torch.Tensor:
        return self.model(x, lengths)
    
    def training_step(self, batch, batch_idx):
        y = batch[-1]
        y_hat = self(*batch[:-1])
        loss = self.loss_fn(y_hat, y.argmax(-1))
        self._train_loss.append(loss)
        self._train_acc(y_hat.argmax(-1), y.argmax(-1))
        return loss

    def validation_step(self, batch, batch_idx):
        y = batch[-1]
        y_hat = self(*batch[:-1])
        loss = self.loss_fn(y_hat, y.argmax(-1))
        self._valid_loss.append(loss)
        self._valid_acc(y_hat.argmax(-1), y.argmax(-1))

    def test_step(self, batch, batch_idx):
        y = batch[-1]
        y_hat = self(*batch[:-1])
        loss = self.loss_fn(y_hat, y.argmax(-1))
        self.log('test_loss', loss)

    def on_train_epoch_end(self):
        loss = torch.stack(self._train_loss).mean()
        self.log_dict({'train_loss': loss, 'train_acc': self._train_acc.compute()}, prog_bar=True)
        self._train_loss.clear()
        self._train_acc.reset()

    def on_validation_epoch_end(self):
        loss = torch.stack(self._valid_loss).mean()
        valid_acc = self._valid_acc.compute()
        self._max_valid_acc = max(self._max_valid_acc, valid_acc)
        self.log_dict({'valid_loss': loss, 'valid_acc': valid_acc, 'max_valid_acc': self._max_valid_acc}, prog_bar=True)
        self._valid_loss.clear()
        self._valid_acc.reset()

    def configure_optimizers(self):
        optim = torch.optim.AdamW(
            self.parameters(), 
            lr=self.hparams.lr, 
            eps=self.hparams.adam_e, 
            weight_decay=self.hparams.wd
        )
        scheduler = torch.optim.lr_scheduler.OneCycleLR(optim, self.hparams.lr, epochs=self.hparams.epochs, steps_per_epoch=self.hparams.steps_per_epoch, pct_start=self.hparams.pct_start, div_factor=1000, final_div_factor=10000)
        return {"optimizer": optim, "lr_scheduler": scheduler}
        

#### Training routine

Define the function **train_classifier** which describes the training routine for the classifier model using the **Trainer** class of *lightning* 

In [24]:
def lightning_train_classifier(config, logger, *callbacks):
    L.seed_everything(42)
    model = CqaModule(
        Classifier(hidden_dim=config.get("hd_cls"), dropout=config.get("drp")),
        nn.CrossEntropyLoss,
        **config
    )
    trainer = L.Trainer(
        max_epochs=config.get("epochs"),
        accelerator="auto",
        devices=1,
        callbacks=list(callbacks),
        logger=logger,
    )
    trainer.fit(model, train_loader_classifier, valid_loader_classifier)

#### Run parameters

In [25]:
def get_run_name(model_name: str, config: dict):
    params = []
    for k, v in config.items():
        if isinstance(v, float):
            params.append(f"{k}={v:.5f}")
        else:
            params.append(f"{k}={v}")
    return f"{model_name}_{'_'.join(params)}_{time.strftime('%y%m%d%H%M%S')}"

Define Hyperparameters for the run

In [26]:
MODEL_NAME = "Classifier_Baseline"
config = dict(
    lr=1e-4,
    hd_cls=2048,
    epochs=100,
    steps_per_epoch=len(train_loader_classifier),
    adam_e=1e-8,
    drp=0.1,
    wd=0.0,
    pct_start=0.1,
)

#### Checkpointing

#TODO: add documentation

In [27]:
checkpoint_callback = L.pytorch.callbacks.ModelCheckpoint(
    dirpath="./lightning_checkpoints",
    filename=get_run_name(MODEL_NAME, config) + "-{epoch:02d}-{val_acc:.2f}",
    monitor="valid_acc",
    save_last=True,
    mode="max"
)

#### Early Stopping

#TODO: add documentation

In [28]:
early_stopping_callback = L.pytorch.callbacks.EarlyStopping(
    monitor="valid_acc",
    patience=15,
    mode="max",
)

Execute training run

In [29]:
with wandb.init(
    entity="david-schurtenberger",
    project="NLP_Project_1",
    name=get_run_name(MODEL_NAME, config),
    config=config,
) as run:
    wandb_logger = WandbLogger(project="NLP_Project_1")
    lightning_train_classifier(run.config, wandb_logger, checkpoint_callback, early_stopping_callback)

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/jovyan/NLP_FS25/projects/lightning_checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇██
max_valid_acc,▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▅▅▆▆▇▇▇▇▇▇███████
train_acc,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▄▄▄▄▅▅▆▅▆▆▇▆▇▇▇▇███
train_loss,█████████████████████▇▇▇▇▇▇▇▇▆▅▅▅▄▄▃▃▃▂▁
trainer/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇████
valid_acc,▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▁▂▃▂▂▃▂▃▆▄▆▅▇▇▆▇▇▇▇█▅▅▄▄▃
valid_loss,▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▂▂▃▃▄▃▄▅█
epoch,75
max_valid_acc,0.234
train_acc,0.42604
train_loss,1.52887


### Hyperparameter tuning

In [30]:
MODEL_NAME = "Classifier"
def classifier_sweep(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config
        run.name = get_run_name(MODEL_NAME, config)
        checkpoint_callback = L.pytorch.callbacks.ModelCheckpoint(
            dirpath="./lightning_checkpoints",
            filename=get_run_name(MODEL_NAME, config) + "-{epoch:02d}-{val_acc:.2f}",
            monitor="valid_acc",
            save_last=True,
            mode="max"
        )
        wandb_logger = WandbLogger(log_model=True)
        lightning_train_classifier(config, wandb_logger, checkpoint_callback)

In [31]:
sweep_config = {
    "name": "ClassifierSweep",
    "method": "bayes",
    "metric": {"name": "max_valid_acc", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]},
        "hd_cls": {"values": [256, 512, 1024, 2048]},
        "wd": {"min": 0.0, "max": 1e-3},
        "adam_e": {"value": 1e-8},
        "drp": {"min": 0.0, "max": 0.3},
        "pct_start": {"values": [0.1, 0.2, 0.3]},
        "epochs": {"value": 100},
        "steps_per_epoch": {"value": len(train_loader_classifier)},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 10,
        "eta": 2
    }
}

In [32]:
sweep_id = wandb.sweep(
    sweep=sweep_config, 
    entity="david-schurtenberger",
    project="NLP_Project_1",
)

Create sweep with ID: rnfc9zfq
Sweep URL: https://wandb.ai/david-schurtenberger/NLP_Project_1/sweeps/rnfc9zfq


In [ ]:
wandb.agent(sweep_id=sweep_id, function=classifier_sweep, count=50)

wandb: Agent Starting Run: 1vnmzccr with config:
wandb: 	adam_e: 1e-08
wandb: 	drp: 0.2221788261076744
wandb: 	epochs: 100
wandb: 	hd_cls: 1024
wandb: 	lr: 0.08970535131282562
wandb: 	pct_start: 0.2
wandb: 	steps_per_epoch: 18
wandb: 	wd: 0.0009449259188135973
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/jovyan/NLP_FS25/projects/lightning_checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type               | Params | Mode 
----------------------------------------------------------
0 | model      | Classifier         | 1.8 M  | train
1 | loss_fn    | CrossEntropyLoss   | 0      | train
2 | _train_acc | MulticlassAccuracy | 0      | train
3 | _valid_acc | MulticlassAccuracy | 0      | train
-----------------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███
max_valid_acc,▁▂▅▆████████████████████████████████████
train_acc,▁▁▂▂▂▃▃▃▄▄▅▆▇█████████████████▇▆████████
train_loss,██████▇▇▇▇▇▆▆▆▅▅▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▄▃▂▂▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇█████
valid_acc,▃▇██▇▄▅▅▅▆▅▆▆▅▇▄▁▄▂▄▃▅▅▄▂▃▁▅▃▆▄▆▅▃▂▄▂▅▆▄
valid_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▄▄▆▆▇▇▇███
epoch,99
max_valid_acc,0.231
train_acc,0.99886
train_loss,0.00824


wandb: Agent Starting Run: 6a1lsdrv with config:
wandb: 	adam_e: 1e-08
wandb: 	drp: 0.09089384497503972
wandb: 	epochs: 100
wandb: 	hd_cls: 2048
wandb: 	lr: 0.09570188299000544
wandb: 	pct_start: 0.1
wandb: 	steps_per_epoch: 18
wandb: 	wd: 0.0003850684439258692
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


## Evaluation

## Interpretation

# WordEmbeddings &rarr; RNN &rarr; Classifier

## Model

#TODO: add documentation and checkpointing/early stopping

In [26]:
class RNNClassifier(Classifier):
    def __init__(
            self, 
            input_dim: int = 300, 
            hidden_dim_gru: int = 512,
            hidden_dim_linear: int = 4096, 
            output_dim: int = 5, 
            bidirectional: bool = False,
            num_gru_layers: int = 2,
            dropout: float = 0.0,
    ):
        self.num_directions = 2 if bidirectional else 1
        self.num_gru_layers = num_gru_layers
        self.output_dim = output_dim
        super().__init__(
            input_dim=hidden_dim_gru * self.num_directions * self.output_dim, 
            hidden_dim=hidden_dim_linear, 
            output_dim=output_dim,
            dropout=dropout,
        )
        self.gru = nn.GRU(input_dim, hidden_dim_gru, num_layers=2, batch_first=True, bidirectional=bidirectional)
        
    def forward(self, x: torch.Tensor, lengths: torch.Tensor) -> torch.Tensor:
        batch_size, num_choices, seq_len, emb_dim = x.shape
        x = x.view(batch_size * num_choices, seq_len, emb_dim)
        lengths = lengths.to("cpu").type(torch.int64)
        packed_x = nn.utils.rnn.pack_padded_sequence(x, lengths.repeat_interleave(num_choices), batch_first=True)
        h0 = x.data.new_zeros(self.num_gru_layers * self.num_directions, batch_size * num_choices, self.gru.hidden_size)
        _, hn = self.gru(packed_x, h0)
        fnn_in = hn[-self.num_directions:, :, :].transpose(0, 1).flatten(1)
        return super().forward(fnn_in.view(-1, self.output_dim * fnn_in.size(-1)))
    
    def reset_parameters(self):
        super().reset_parameters()
        self.gru.reset_parameters()
        

## Training

#TODO: add documentation

#### Training routine

Define the function **train_rnn_plus_classifier** which describes the training routine for the classifier model using the **Trainer** class of *lightning* 

In [27]:
def train_rnn_plus_classifier(config, logger, *callbacks):
    L.seed_everything(42)
    model = CqaModule(
        RNNClassifier(
            hidden_dim_gru=config.get("hd_gru"),
            hidden_dim_linear=config.get("hd_cls"),
            bidirectional=config.get("bidirectional"),
            dropout=config.get("drp"),
        ),
        nn.CrossEntropyLoss,
        **config
    )
    trainer = L.Trainer(
        max_epochs=config.get("epochs"),
        accelerator="auto",
        devices=1,
        callbacks=list(callbacks),
        logger=logger,
    )
    trainer.fit(model, train_loader_rnn, valid_loader_rnn)

Define run parameters

In [28]:
MODEL_NAME = "RNN+Classifier_Baseline"
config = dict(
    lr=1e-4,
    hd_cls=2048,
    hd_gru=128,
    bidirectional=False,
    epochs=100,
    steps_per_epoch=len(train_loader_rnn),
    adam_e=1e-8,
    drp=0.1,
    wd=0.0,
    pct_start=0.1,
)

#### Checkpointing

#TODO: add documentation

In [29]:
checkpoint_callback = L.pytorch.callbacks.ModelCheckpoint(
    dirpath="./lightning_checkpoints",
    filename=get_run_name(MODEL_NAME, config) + "-{epoch:02d}-{val_acc:.2f}",
    monitor="valid_acc",
    save_last=True,
    mode="max"
)

#### Early stopping

#TODO: add documentation

In [30]:
early_stopping_callback = L.pytorch.callbacks.EarlyStopping(
    monitor="valid_acc",
    patience=15,
    mode="max"
)

Initialize wandb experiment tracking for run


In [ ]:
with wandb.init(
    entity="david-schurtenberger",
    project="NLP_Project_1",
    name=get_run_name(MODEL_NAME, config),
    config=config,
) as run:
    wandb_logger = WandbLogger(project="NLP_Project_1")
    train_rnn_plus_classifier(run.config, wandb_logger, checkpoint_callback, early_stopping_callback)

### Hyperparameter tuning

In [31]:
MODEL_NAME = "RNN+Classifier"
def rnn_sweep(config=None):
    with wandb.init(config=config) as run:
        config = wandb.config
        run.name = get_run_name(MODEL_NAME, config)
        checkpoint_callback = L.pytorch.callbacks.ModelCheckpoint(
            dirpath="./lightning_checkpoints",
            filename=get_run_name(MODEL_NAME, config) + "-{epoch:02d}-{val_acc:.2f}",
            monitor="valid_acc",
            save_last=True,
            mode="max"
        )
        wandb_logger = WandbLogger(log_model=True)
        train_rnn_plus_classifier(config, wandb_logger, checkpoint_callback)

In [34]:
sweep_config = {
    "name": "RNN+ClassifierSweep",
    "method": "bayes",
    "metric": {"name": "max_valid_acc", "goal": "maximize"},
    "parameters": {
        "lr": {"values": [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]},
        "hd_cls": {"values": [256, 512, 1024, 2048, 4096]},
        "hd_gru": {"values": [32, 64, 128, 256]},
        "bidirectional": {"values": [True, False]},
        "wd": {"min": 0.0, "max": 1e-3},
        "adam_e": {"value": 1e-8},
        "drp": {"min": 0.0, "max": 0.3},
        "pct_start": {"values": [0.1, 0.2, 0.3]},
        "epochs": {"value": 100},
        "steps_per_epoch": {"value": len(train_loader_classifier)},
    },
    "early_terminate": {
        "type": "hyperband",
        "min_iter": 10,
        "eta": 2
    }
}

In [35]:
sweep_id = wandb.sweep(
    sweep=sweep_config, 
    entity="david-schurtenberger",
    project="NLP_Project_1",
)

Create sweep with ID: mo76tgs8
Sweep URL: https://wandb.ai/david-schurtenberger/NLP_Project_1/sweeps/mo76tgs8


In [ ]:
wandb.agent(sweep_id=sweep_id, function=rnn_sweep, count=50)

wandb: Agent Starting Run: tsx7nhhv with config:
wandb: 	adam_e: 1e-08
wandb: 	bidirectional: False
wandb: 	drp: 0.08956409167961028
wandb: 	epochs: 100
wandb: 	hd_cls: 4096
wandb: 	hd_gru: 64
wandb: 	lr: 0.05537328328677624
wandb: 	pct_start: 0.2
wandb: 	steps_per_epoch: 18
wandb: 	wd: 0.0008623167176081347
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A16') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/opt/conda/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /home/jovyan/NLP_FS25/projects/lightning_checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (18) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Evaluation

## Interpretation